# Lien vers la base de donnee : 
https://www.kaggle.com/datasets/kartmaan/dictionnaire-francais

# L'importatoin des librairies

In [128]:
import pandas as pd

import mysql.connector

import json

# Une consultation du dictionnaire

In [105]:
df = pd.read_csv("dico.csv")
df

,Mot,Définitions
0,Accueil,"[""Cérémonie ou prestation réservée à un nouvel..."
1,Dictionnaire,['Ouvrage de référence qui répertorie des mots...
2,Lire,['Interpréter des informations écrites sous fo...
3,Encyclopédie,"['Ensemble, enchaînement de toutes les connais..."
4,Manga,"['Bande dessinée japonaise, souvent en noir et..."
...,...,...
973666,Sévice,['mauvais traitement.']
973667,Faire voir,"['Montrer, faire connaître.']"
973668,Laisser voir,"['Montrer, faire connaître.']"
973669,Coronavirus,['Genre de virus à ARN de la famille des coron...


# Creation de la base de donnee dictionaire

In [119]:

db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',

}

try:
    # Establishing the connection
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    # SQL queries to create database and table
    create_db_query = "CREATE DATABASE IF NOT EXISTS dictionaire;"

    # Executing database and table creation queries
    cursor.execute(create_db_query)
    print('Base de donnee cree avec succes')

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    # Closing the cursor and connection
    cursor.close()
    conn.close()

Base de donnee cree avec succes


# Creation de la table Mots

In [120]:
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'dictionaire',
}

try:
    # Establishing the connection
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    create_table_query = """
    CREATE TABLE IF NOT EXISTS Mots (
        mot VARCHAR(255) NOT NULL,
        definition TEXT NOT NULL
    );
    """
    cursor.execute(create_table_query)
    print('Table Mots cree avec succes')
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    # Closing the cursor and connection
    cursor.close()
    conn.close()


Table Mots cree avec succes


# Importer notre csv a la table Mots

In [121]:

try:
    # Establishing the connection
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    for i in range(len(df)):
        mot = str(df.loc[i, 'Mot'])
        mot = mot.capitalize()
        definitions = df.loc[i, 'Définitions']
        
        # Ensure the mot and definitions are not NaN
        if pd.isna(mot) or pd.isna(definitions):
            print(f"Skipping row {i} due to NaN values.")
            continue
        
        # Convert the definitions list to a JSON string
        definitions_json = json.dumps(definitions)
        
        query = "INSERT INTO Mots (mot, definition) VALUES (%s, %s);"
        cursor.execute(query, (mot, definitions_json))

    # Committing the transaction
    conn.commit()
    
    print('La table a été chargée avec succès')
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    # Closing the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()

La table a été chargée avec succès


# Example d'usage

In [134]:
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'dictionaire',
}

def get_definition(mot):
    try:
        mot = mot.capitalize()
        # Establishing the connection
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        query = "SELECT definition FROM Mots WHERE mot = %s;"
        cursor.execute(query, (mot,))

        result = cursor.fetchone()
        
        if result:
            definitions_json = result[0]
            definitions = json.loads(definitions_json)  # Convert JSON string back to list
            definitions = eval(definitions)
            return definitions
        else:
            return f"Le mot '{word}' n'a pas été trouvé dans la base de données."

    except mysql.connector.Error as err:
        return f"Error: {err}"
    finally:
        # Closing the cursor and connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Example usage
word = "salut"
definitions = get_definition(word)

for definition in definitions :
    print(definition)


Conservation ou rétablissement dans un état heureux, dans un état convenable.
Cessation de danger, recouvrement de la sécurité.
Fait d'échapper à la damnation, de parvenir à la félicité éternelle.
Salut du Saint Sacrement, qui est une cérémonie qui comporte essentiellement la bénédiction du saint sacrement.
Action de saluer.
Bonjour.
Au revoir.
Formule exclamative de souhait, de civilité, employée aussi dans le préambule des lois et ordonnances, dans les lettres patentes des rois, dans les bulles des papes, dans les mandements des archevêques et évêques, etc.
